<a href="https://colab.research.google.com/github/WolfDev8675/RepoSJX7/blob/Assign5_1/Code_proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Board Game Review Project***

###*Primaries*

In [ ]:
#!usr/bin/python

''' Board Game Review Project ''' 
# 

#imports 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plts
import warnings 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge as R2
from sklearn.linear_model import BayesianRidge as BR2
from sklearn.svm import SVR 
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFT

###*Presetting viewer*

In [ ]:
# Presetting viewer 
pd.set_option('display.max_columns', None)

##*Loading dataset: data-specs, size and contents*

In [ ]:
# Loading dataset: data-specs, size and contents
d_set=pd.read_csv("https://raw.githubusercontent.com/WolfDev8675/RepoSJX7/Assign5_1/data/games.csv")
#print(' Dataset example: \n')
d_set.head(20)

In [ ]:
print('_'.center(70,'_'))
print('\n\n Content information :: \n');d_set.info();print('_'.center(70,'_'));
print('\n');print('x'.center(70,'*'))

##*Noise and Vaccancy Study*

In [ ]:
# Noise and Vaccancy Study 
print(" Noise and vacancy study ");print('_'.center(70,'_'));
print(' Null values per column \n\n',d_set.isnull().sum());print('_'.center(70,'_'))   #8  fields has nulls 
print(' Connection between nulls of various fields:-> [Name] \n',d_set[d_set['name'].notna()].isnull().sum());print('_'.center(70,'_'))
print(""" Connection between nulls of various fields:-> [yearpublished, minplayers, 
        maxplayers, playingtime, minplaytime, maxplaytime, minage] \n""",
      d_set[d_set['yearpublished'].isnull()])
print('_'.center(70,'_'));
print('x'.center(70,'*'))
# ---- conclusion: nulls in name field is solo others are connected 


##*Clearing dataset: noise and vaccancy cleaning*

In [ ]:
# Clearing dataset: noise and vaccancy cleaning 
d_set_f0=d_set.dropna(axis='index'); # separately removes 41+3 index lines 
print(' Pre-Cleaning size  : ',d_set.shape);print('_'.center(70,'_'));
print(' Post-Cleaning size : ',d_set_f0.shape);print('_'.center(70,'_'));


##*Exploration of the data*

In [ ]:
# Exploration of the data
# EDA:#1
fig_1=plts.figure(figsize=(13,6.5));fig_1.canvas.set_window_title('EDA plot(1): Average rating histogram')
sns.histplot(d_set_f0['average_rating']);
plts.xlabel('Average Rating');plts.title('Variation of average rating received');
plts.show();

In [ ]:
# EDA:#2
d_set_f0[d_set_f0['average_rating']==0].head()    #check:1

In [ ]:
# EDA:#2(contd.)
d_set_f0[d_set_f0['average_rating']>0].head()    #check:2

info determined: games with average_rating =0 also has users_rating =0. Hence, all those games that has no average rating can be contributed to also having no users rating for the game and therefore may not be a candidate to analysis.

In [ ]:
# EDA decision #1 (EDA#1 & EDA#2)
d_set_f1=d_set_f0[d_set_f0['average_rating']>0] # keeping only the games that have at least an average rating 
# EDA:#3
fig_2=plts.figure(figsize=(13,6.5));fig_2.canvas.set_window_title('EDA plot(2): Average rating histogram')
sns.histplot(d_set_f1['average_rating']);
plts.xlabel('Average Rating');plts.title('Variation of average rating received');
plts.show();

In [ ]:
# EDA:#4
corr_info=d_set_f1.corr()
fig_3=plts.figure(figsize=(13,11));fig_3.canvas.set_window_title('EDA plot(3): Field Correlation information')
sns.heatmap(corr_info,vmax=0.8,square=True);plts.xticks(rotation=45,ha='right');
plts.xlabel('Fields');plts.ylabel('Fields');plts.title('Correlation between fields of the dataset ');
plts.show();

info determined: average rating is the target variable for determination of a game quality.Secondarily, the name, type and id fields have no contribution. Also bayes rating and publishing year have very low correlation to the determining factors.

In [ ]:
# EDA decision #2 (EDA#3 & EDA#4)
contributors=[a_column for a_column in d_set_f1.columns.tolist() if a_column not in ['id','name','type','bayes_average_rating','yearpublished']]
affectors='average_rating'
full_X=d_set_f1[contributors].values
full_Y=d_set_f1[affectors].values
print(' Dataset components (After EDA decision: #2 ) : ');print('_'.center(70,'_'));
print(" Independent variables' value set \n",full_X);print('_'.center(70,'_'));
print(" Dependent variable's value set \n",full_Y);print('_'.center(70,'_'));

##*Visualization of the fields distribution*

In [ ]:
fig_4=plts.figure(figsize=(20,14.5));fig_4.canvas.set_window_title('Visuals plot(1): Visualization scatters of contributing fields ')
pos_counter=1 #preset value for 16 places 4x4 setup 
warnings.filterwarnings('ignore')
plts.title('Comparison of Average rating with other field values ');
plts.tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False)
plts.tick_params(axis='y',which='both',left=False,right=False,labelleft=False)
plts.gca().spines['left'].set_visible(False)
plts.gca().spines['right'].set_visible(False)
plts.gca().spines['bottom'].set_visible(False)
plts.gca().spines['top'].set_visible(False)
for a_column in contributors:
    fig_4.add_subplot(4,4,pos_counter)
    sns.scatterplot(d_set_f1[a_column],d_set_f1[affectors],alpha=0.5)
    plts.xlabel(a_column);plts.ylabel(affectors);
    pos_counter+=1
plts.show()

##*Training on Models*

###*Part:1 Splitting Data*

In [ ]:
# Training on Models 
#Part:1 Splitting 
trainX,testX,trainY,testY=train_test_split(full_X,full_Y,test_size=0.21,random_state=52)
print(' Training set size for Independent variables  : ',trainX.shape);print('_'.center(70,'_'));
print(' Training set size for Dependent variables    : ',trainY.shape);print('_'.center(70,'_'));
print(' Testing set size for Independent variables   : ',testX.shape);print('_'.center(70,'_'));
print(' Testing set size for Dependent variables     : ',testY.shape);print('_'.center(70,'_'));

###*Part:2 Model Creation (Multiple models)*

In [ ]:
#Part:2 Model Creation (Multiple models)
model_list="""
        Linear Regression         : LR
        Ridge Regression          : R2
        Bayesian Ridge Regression : BR2
        Support Vector Regression : SVR
        Decision Tree Regression  : DTR
        Random Forest Regression  : RFT
        """
print('Creating models for : \n',model_list);print('_'.center(70,'_'));
models={'LR':LR(),'R2':R2(),'BR2':BR2(),'SVR':SVR(),'DTR':DTR(),'RFT':RFT()}

###*Part:3 Training Data on Models (Multiple Models)*


In [ ]:
#Part:3 Training Data on Models (Multiple models)
print(' Model Training Status : ')
for key in models:
    models[key].fit(trainX,trainY)
    print('\tTrained Model : ',key)
print('_'.center(70,'_'));